# Tiny few shot classifer

Im folgenden wird ein few shot classifier für multilabels im [SetFit-Frame](https://huggingface.co/blog/setfit) Format erzeugt. Gearbeitet wird mit `Python 3.12.3`.

# Pakete laden

In einem ersten Schritt werden die nötigen Pakete geladen. 

In [1]:
from datasets import load_dataset  # Zum Laden von Datensätzen aus der Hugging Face `datasets` Bibliothek
from sentence_transformers.losses import CosineSimilarityLoss  # Verlustfunktion basierend auf der Kosinusähnlichkeit, für Aufgaben wie Textähnlichkeit

from setfit import SetFitModel, SetFitTrainer  # Zum Laden des SetFit-Modells und Trainers, speziell für Few-Shot Learning mit Satztransformern
import pandas as pd  # Pandas für Datenmanipulation und -analyse
from sklearn.preprocessing import LabelEncoder  # Zum Encoden von Labels in numerische Werte für maschinelles Lernen
import os  # Für den Umgang mit Betriebssystemfunktionen (z.B. Dateipfade)
from datasets import Dataset, DatasetDict  # Zum Erstellen und Verwalten von Datensätzen in Hugging Face `datasets`-Format
from sklearn.model_selection import train_test_split  # Zum Aufteilen der Daten in Trainings- und Testsets

from transformers import TrainingArguments  # Zum Festlegen von Trainingsparametern für Modelle in der Transformers-Bibliothek

import torch  # Ermöglicht die GPU-Nutzung und effiziente Verarbeitung von Tensoren
import numpy as np  # Für numerische Berechnungen und Arbeiten mit Arrays

import re
import html

from unidecode import unidecode

# Arbeitsverzeichnis

Es wird weiterhin das Rootverzeichnis bestimmt.

In [2]:
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification\Py


# Daten

## Daten laden

Das zur Zeit keine Daten für das trainieren eines multilable Classifiers vorliegen, werden die Trainings-Daten genutzt, die ursprünglich von Franziska Weber und Felix Süßenbach erzeugt wurden.

In [3]:
# Laden der Excel-Datei
df = pd.read_excel('data/train_data_with_negative_cases.xlsx')
 
# 1. Ersetze NaN-Werte durch 0
df = df.fillna(0)

# 2. Ersetze unendliche Werte durch 0
df.replace([np.inf, -np.inf], 0, inplace=True)

df[df.select_dtypes(include='number').columns] = df.select_dtypes(include='number').astype('int64')

 
df_cleaned = df[~(df.select_dtypes(include='number').eq(0).all(axis=1))]



# Für die weitere Verarbeitung werden die Daten ins  Hugging Face Dataset-Format transformiert. 

# /////////////////////////////////////////////////
# Hinweis: Das Dataset-Format der Hugging Face datasets-Bibliothek basiert auf Apache Arrow und ist speichereffizient, was schnelles Laden und Verarbeiten großer Datensätze ermöglicht. Es integriert sich nahtlos in Machine-Learning-Workflows und unterstützt einfache Transformationen und Vorverarbeitung durch Methoden wie .map(). Zudem ist es flexibel und kompatibel mit Pandas DataFrames sowie gängigen Frameworks wie PyTorch und TensorFlow.
#//////////////////////////////////////////////////

dataset = Dataset.from_pandas(df)
print(dataset)
 

Dataset({
    features: ['Hardware-/Robotikentwicklung', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'Data Analytics & KI', 'Quantencomputing', 'IT-Architektur', 'sentence'],
    num_rows: 362
})


## Daten verarbeiten

### Feature Auswahl

In einem ersten Schritt werden die Features des Classifiers ausgewählt. Da sich insbesondere *klassische* und *transformative Future Skills* als schwer nachvollziehbar in den Daten erwiesen haben, werden diese aus dem Classifier ausgeschlossen, ebenso wie die Kurstitel und -beschreibungen (`sentence`). 

In [4]:
# ziehe column names als Liste
features = dataset.column_names

# definiere skills, die nicht vom classifier berücksichtigt werden
to_remove = ['sentence']

# schließe die oben definierten Variablen aus der Feature-Liste aus
filtered_features = [feature for feature in features if feature not in to_remove]

features = filtered_features

print(features)

['Hardware-/Robotikentwicklung', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'Data Analytics & KI', 'Quantencomputing', 'IT-Architektur']


### One-Hot-Encoding

Für das Trainieren des Classifiers wird im folgenden ein *One-Hot-Encoding*-Array der Features erzeugt. Dies ist deshalb nötig, da im folgenden ein Modell für ein Multi-Label-Szenario trainiert werden soll: Ein Kursangebot kann den erwerb mehrerer Future Skills versprechen, etwa Softwareentwicklung **&** IT-Architektur. Das One-Hot-Encoding wird benötigt, um die Labels der Features in eine Form zu bringen, die das Classifier Modell von SetFit korrekt verarbeiten kann. Es stellt sicher, dass alle Klassen gleichwertig behandelt werden, und ermöglicht die Berechnung von Ähnlichkeiten verschiedener Kursinhalte.

Um den *One-Hot-Encoding*-Array zu erzeugen definieren wir `encode_labels(record)`: Diese Funktion geht durch jede Zeile eines Data Frames und erstellt eine neue Spalte namens `labels`. In dieser Spalte werden die Werte aus den oben definierten "features" gesammelt und in eine Liste gepackt. Diese Liste enthält Werte der Features.

Beispiel: Mit Blick auf die Features 'Data Analytics & KI', 'Softwareentwicklung' und 'Nutzerzentriertes Design', könnte ein Zeilenvektor des Arrays eines Kurses der sich mit Data Analytics & KI / Softwareentwicklung beschäftigt so aussehen: `[1, 1, 0]`

In [5]:
# generiere Funktion für One-Hot-Encoding für jede Zeile (record)
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}

# wende Funktion auf train_data an
dataset = dataset.map(encode_labels)

# um die Daten übersichtlich im viewer zu betrachten: umwandlung und pandas data frame
pd_dataset = dataset.to_pandas()

c:\Users\Hueck\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\utils\_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/362 [00:00<?, ? examples/s]

### Erstellung von Test- und Trainingsdaten

Im folgenden wird in `dataset` für jedes Feature (`f`) in der Liste `features` nach den Zeilen gesucht, in denen das Feature einen gültigen (nicht-leeren oder nicht-null) Wert hat. Aus diesen Zeilen (Indizes) werden dann zufällig 8 Beispiele ausgewählt. `np.where(dataset[f])` gibt die Indizes der Zeilen zurück, bei denen der Wert in der Spalte `f` `true` ist (also weder leer noch null). Wenn der Wert `false` ist (z.B. `0`, `NaN`, `None`, `False`, ein leerer String ''), wird diese Zeile ausgeschlossen.

In [7]:
# num_samples = 8
# samples = np.concatenate(
#     [np.random.choice(np.where(dataset[f])[0], num_samples) for f in features]
# )
# print(samples)

Im Folgenden wird einerseits der Trainingsdatensatz `train_dataset` anhand der oben definierten Indizes aus dem `samples`-Objekt erzeugt. Andererseits wird ein komplementärer Datensatz `eval_dataset` erzeugt, mit allen Indizes die nicht im `samples`-Objekt vorkommen. 

In [6]:
train_dataset = dataset
# eval_dataset = dataset.select(
#     np.setdiff1d(np.arange(len(dataset)), samples)
# )

 
# check data with pandas
pd_train_dataset = train_dataset.to_pandas()
# pd_eval_dataset = eval_dataset.to_pandas()


In [7]:
import torch
torch.cuda.empty_cache()

## Modelltraining

Es wird das vortrainierte sentence-transformers-Modell `paraphrase-mpnet-base-v2` aus der Hugging Face Bibliothek geladen. Durch die Angabe der Strategie `one-vs-rest` wird das Modell für eine Multi-Label-Klassifikation konfiguriert, bei der jede Klasse eines Kurses einzeln vorhergesagt wird. Auf diese Weise wird es möglich, mehrere Skills pro Kurs zu identifizieren. 

In [24]:
# Laden des Modells
model_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SetFitModel.from_pretrained(model_id, multi_target_strategy="multi-output")

# Manuelles Hinzufügen der Label-Konfiguration
model.__dict__["label2id"] = {
    'Hardware-/Robotikentwicklung': 0,
    'Softwareentwicklung': 1,
    'Nutzerzentriertes Design': 2,
    'Data Analytics & KI': 3,
    'Quantencomputing': 4,
    'IT-Architektur': 5,
}



model.__dict__["id2label"] = {v: k for k, v in model.__dict__["label2id"].items()}

# Bestätigen, dass die Labels korrekt hinzugefügt wurden
print("Label zu ID Mapping:", model.__dict__["label2id"])
print("ID zu Label Mapping:", model.__dict__["id2label"])

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Label zu ID Mapping: {'Hardware-/Robotikentwicklung': 0, 'Softwareentwicklung': 1, 'Nutzerzentriertes Design': 2, 'Data Analytics & KI': 3, 'Quantencomputing': 4, 'IT-Architektur': 5}
ID zu Label Mapping: {0: 'Hardware-/Robotikentwicklung', 1: 'Softwareentwicklung', 2: 'Nutzerzentriertes Design', 3: 'Data Analytics & KI', 4: 'Quantencomputing', 5: 'IT-Architektur'}


GPU aktieviere

In [25]:
import torch

# Prüfen, ob CUDA (GPU) verfügbar ist
print("CUDA verfügbar:", torch.cuda.is_available())

# Wenn CUDA verfügbar ist, die GPU verwenden
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Verwende CPU")

# model auf GPU verschieben

    # Sicherstellen, dass das Modell auf der GPU ist, wenn verfügbar
model = model.to(device)



CUDA verfügbar: True
GPU: NVIDIA GeForce RTX 4060 Ti


Der `SetFitTrainer` wird instanziiert, um das Modell auf den Trainings- und Evaluierungsdatensätzen zu trainieren. Eine Bewertung des Modells wird erst zu einem späteren Zeitpunkt möglich, wenn Testdaten vorliegen.

1. Das vortrainierte `model` (siehe oben) wird als Grundlage für das Training verwendet, das auf dem Few-Shot Learning-Ansatz basiert. `paraphrase-mpnet-base-v2` ist ein vortrainiertes Modell aus der Sentence Transformers-Familie, das speziell für die Aufgabe der semantischen Textähnlichkeit und Paraphrasenerkennung entwickelt wurde. 
2. Der `train_dataset` enthält die Trainingsdaten, und der `eval_dataset` wird zu einem späteren Zeitpunkt genutzt, um die Klassifikations-Leistung des Modells während des Trainings zu bewerten.
3. Die Verlustfunktion (`loss_class`) wird als *Cosine Similarity Loss* festgelegt. Die *Cosine Similarity Loss* ist eine Verlustfunktion, die darauf basiert, wie ähnlich sich zwei Vektoren sind. In diesem Fall werden die Ausgabevektoren des Modells und die Zielvektoren (also die "Labels") mithilfe der Kosinus-Ähnlichkeit verglichen. Je höher die Ähnlichkeit also, desto besser das Modell. 
4. Die `column_mapping` gibt an, welche Spalten in den Datensätzen die Texte (`sentence` → `text`) und die zugehörigen Labels (`labels` → `label`) enthalten, sodass das Modell die Daten korrekt interpretieren kann.


In [26]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset, 
    #eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=40, 
    num_epochs=2,
    column_mapping={"sentence": "text", "labels": "label"}
)

C:\Users\Hueck\AppData\Local\Temp\ipykernel_7592\2679062307.py:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
c:\Users\Hueck\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\utils\_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/362 [00:00<?, ? examples/s]

Nun kann das Training beginnen: 

In [27]:
trainer.train()

***** Running training *****
  Num unique pairs = 28960
  Batch size = 16
  Num epochs = 2


  0%|          | 0/3620 [00:00<?, ?it/s]

{'embedding_loss': 0.2218, 'grad_norm': 2.095167398452759, 'learning_rate': 5.524861878453039e-08, 'epoch': 0.0}
{'embedding_loss': 0.2162, 'grad_norm': 1.7631800174713135, 'learning_rate': 2.7624309392265196e-06, 'epoch': 0.03}
{'embedding_loss': 0.171, 'grad_norm': 1.5138044357299805, 'learning_rate': 5.524861878453039e-06, 'epoch': 0.06}
{'embedding_loss': 0.1561, 'grad_norm': 1.0122721195220947, 'learning_rate': 8.287292817679558e-06, 'epoch': 0.08}
{'embedding_loss': 0.1432, 'grad_norm': 2.277106285095215, 'learning_rate': 1.1049723756906078e-05, 'epoch': 0.11}
{'embedding_loss': 0.1406, 'grad_norm': 1.8083763122558594, 'learning_rate': 1.3812154696132598e-05, 'epoch': 0.14}
{'embedding_loss': 0.1296, 'grad_norm': 1.8593485355377197, 'learning_rate': 1.6574585635359117e-05, 'epoch': 0.17}
{'embedding_loss': 0.1127, 'grad_norm': 1.8592171669006348, 'learning_rate': 1.9337016574585635e-05, 'epoch': 0.19}
{'embedding_loss': 0.1001, 'grad_norm': 0.930862545967102, 'learning_rate': 1.9

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'embedding_loss': 0.0848, 'grad_norm': 1.2511671781539917, 'learning_rate': 1.884591774094537e-05, 'epoch': 0.3}
{'embedding_loss': 0.0698, 'grad_norm': 1.348893642425537, 'learning_rate': 1.8538980969920198e-05, 'epoch': 0.33}
{'embedding_loss': 0.075, 'grad_norm': 1.6110608577728271, 'learning_rate': 1.823204419889503e-05, 'epoch': 0.36}
{'embedding_loss': 0.0721, 'grad_norm': 0.7072257995605469, 'learning_rate': 1.7925107427869862e-05, 'epoch': 0.39}
{'embedding_loss': 0.0692, 'grad_norm': 1.486924409866333, 'learning_rate': 1.761817065684469e-05, 'epoch': 0.41}
{'embedding_loss': 0.056, 'grad_norm': 1.3084622621536255, 'learning_rate': 1.7311233885819523e-05, 'epoch': 0.44}
{'embedding_loss': 0.0626, 'grad_norm': 0.979653537273407, 'learning_rate': 1.7004297114794355e-05, 'epoch': 0.47}
{'embedding_loss': 0.0712, 'grad_norm': 1.7086412906646729, 'learning_rate': 1.6697360343769183e-05, 'epoch': 0.5}
{'embedding_loss': 0.062, 'grad_norm': 1.352835774421692, 'learning_rate': 1.63904

...

{'embedding_loss': 0.0455, 'grad_norm': 1.0911186933517456, 'learning_rate': 3.4990791896869245e-06, 'epoch': 1.69}
{'embedding_loss': 0.044, 'grad_norm': 0.6668357849121094, 'learning_rate': 3.192142418661756e-06, 'epoch': 1.71}
{'embedding_loss': 0.0464, 'grad_norm': 0.9320376515388489, 'learning_rate': 2.885205647636587e-06, 'epoch': 1.74}
{'embedding_loss': 0.046, 'grad_norm': 0.7753127217292786, 'learning_rate': 2.578268876611418e-06, 'epoch': 1.77}
{'embedding_loss': 0.0417, 'grad_norm': 0.8578892946243286, 'learning_rate': 2.2713321055862494e-06, 'epoch': 1.8}
{'embedding_loss': 0.0449, 'grad_norm': 0.7696364521980286, 'learning_rate': 1.9643953345610806e-06, 'epoch': 1.82}
{'embedding_loss': 0.0435, 'grad_norm': 1.135198712348938, 'learning_rate': 1.6574585635359118e-06, 'epoch': 1.85}
{'embedding_loss': 0.04, 'grad_norm': 0.6484357714653015, 'learning_rate': 1.3505217925107427e-06, 'epoch': 1.88}
{'embedding_loss': 0.0452, 'grad_norm': 0.48058751225471497, 'learning_rate': 1.0

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'embedding_loss': 0.0468, 'grad_norm': 0.5405465364456177, 'learning_rate': 4.297114794352364e-07, 'epoch': 1.96}
{'embedding_loss': 0.0397, 'grad_norm': 0.7114013433456421, 'learning_rate': 1.2277470841006753e-07, 'epoch': 1.99}


Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'train_runtime': 541.5306, 'train_samples_per_second': 106.956, 'train_steps_per_second': 6.685, 'train_loss': 0.06277019726884299, 'epoch': 2.0}


In [28]:
# Speichern des Modells auf deinem PC
model.save_pretrained("models")  # Ersetze den Pfad durch deinen gewünschten Speicherort

In [15]:
# # Modell evaluieren (Nur bei vorhandensein von Testdaten)
# metrics = trainer.evaluate()
# metrics


In [21]:
trainer.push_to_hub("Chernoffface/fs-setfit-multilable-model")

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Chernoffface/fs-setfit-multilable-model/commit/587080c871a8bd3e090274308632a0a2e20cabfd', commit_message='Add SetFit model', commit_description='', oid='587080c871a8bd3e090274308632a0a2e20cabfd', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("Chernoffface/fs-setfit-multilable-model")

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/305 [00:00<?, ?B/s]

config_setfit.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

In [23]:
preds = model(
    [
        "Automation Projects, Automation Methods, Development Methods for Automation Systems, Automation with Qualitative Models, Safety and Reliability of Automation Systems",
        "Inhalt: I) Grundlagen - Motivation - Grundelemente der Quantenmechanik - Erinnerung II) Streuung in der Quantenmechanik - Formulierung des Streuproblems - Partialwellenentwicklung und Streuobservablen - Näherungsmethoden III) Quantenmechanisches Vielteilchenproblem - Unterscheidbare vs. identische Teilchen - Symmetrische und antisymmetrische Vielteilchenzustände - Zweite Quantisierung - Exakte Lösungen und Näherungsmethoden - Hartree-Fock & Gross-Pitaevskii Näherung IV) Relativistische Quantenmechanik - Erinnerung an die spez.",
        "Die Themen umfassen die Architektur moderner Spielesysteme, sowohl Hard- als auch Software, das Zeitmanagement im Millisekundenbereich, das Laden und Komprimieren von Assets sowie das physikalisch basierte Echtzeit-Rendering und Animationen. Weitere Schwerpunkte sind der Umgang mit großen Spielwelten, die Audiomodellierung und -mischung, physikbasierte Simulationen mittels Constraints sowie der Einsatz von künstlicher Intelligenz in Spielen. Zudem werden die Netzwerkkommunikation in Multiplayer-Spielen, die prozedurale Erstellung von Inhalten, die Integration von Skriptsprachen und die Optimierung sowie Parallelisierung von CPU- und GPU-Code behandelt. Die Übungen kombinieren dabei theoretische Grundlagen mit praktischen Anwendungen.",
        "Im Mittelpunkt des Seminars steht die neue Renaissance-Poesie, die aus Italien kommend die traditionelle spanische Lyrik zurückdrängt. Dichtungsformen und Inhalte werden beschrieben und die Wirkung auf die Literatur des siglo de oro festgehalten. Textausgaben: Juan Boscán, Obra completa, ed. Carlos Clavería, Madrid: Cátedra,  ISBN 978-84-376-1685-8 Garcilaso de la Vega, Poesia castellana completa, Madrid: Cátedra ISBN 978-84-376-0067-3 Die Texte sollen angeschafft werden. Weitere Texte werden gegebenenfalls vervielfältigt."
    ]
)
[[f for f, p in zip(features, ps) if p] for ps in preds]

[['Hardware-/Robotikentwicklung'],
 ['Quantencomputing'],
 ['Softwareentwicklung', 'Data Analytics & KI'],
 []]